# Quantum teleporation

**Reference**  
https://github.com/quantumlib/Cirq/blob/master/examples/quantum_teleportation.py  
https://en.wikipedia.org/wiki/Quantum_teleportation  
https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.70.1895  

Quantum Teleportation is a process by which a quantum state can be transmitted
by sending only two classical bits of information. This is accomplished by
pre-sharing an entangled state between the sender (Alice) and the receiver
(Bob). This entangled state allows the receiver (Bob) of the two classical
bits of information to possess a qubit with the same state as the one held by
the sender (Alice).
In the following example output, qubit 0 (the Message) is set to a random state
by applying X and Y gates. By sending two classical bits of information after
qubit 0 (the Message) and qubit 1 (Alice's entangled qubit) are measured, the
final state of qubit 2 (Bob's entangled qubit) will be identical to the
original random state of qubit 0 (the Message). This is only possible given
that an entangled state is pre-shared between Alice and Bob.

In [1]:
import random
import numpy as np
import cirq

In [3]:
ranX = random.random()
ranY = random.random()

In [36]:
msg, alice, bob = cirq.LineQubit.range(3)

In [37]:
circuit = cirq.Circuit()

In [38]:
# Creates Bell state to be shared between Alice and Bob
circuit.append([cirq.H(alice), cirq.CNOT(alice, bob)])    

In [39]:
# Creates a random state for the Message
circuit.append([cirq.X(msg)**ranX, cirq.Y(msg)**ranY], strategy=cirq.InsertStrategy.NEW_THEN_INLINE)

In [40]:
# Bell measurement of the Message and Alice's entangled qubit
circuit.append([cirq.CNOT(msg, alice), cirq.H(msg)])
circuit.append(cirq.measure(msg, alice))    

In [41]:
# Uses the two classical bits from the Bell measurement to recover the
# original quantum Message on Bob's entangled qubit
circuit.append([cirq.CNOT(alice, bob), cirq.CZ(msg, bob)])

In [42]:
print(circuit)

0: ───────────X^0.337───Y^0.069───@───H───M───────@───
                                  │       │       │
1: ───H───@───────────────────────X───────M───@───┼───
          │                                   │   │
2: ───────X───────────────────────────────────X───@───


In [43]:
# Compare with the original message
msg0 = cirq.NamedQubit('msg')
circuit0 = cirq.Circuit([cirq.X(msg0)**ranX, cirq.Y(msg0)**ranY])
print(circuit0)

msg: ───X^0.337───Y^0.069───


In [44]:
sim = cirq.Simulator()

In [45]:
result0 = sim.simulate(circuit0)

In [46]:
# Bloch vector for the original message
cirq.bloch_vector_from_state_vector(result0.final_state, 0)

array([ 0.10487789, -0.87166107,  0.47875637], dtype=float32)

In [58]:
result = sim.simulate(circuit)

In [59]:
result.measurements

{'0,1': array([0, 1], dtype=uint8)}

In [48]:
# Bloch vector for the teleported message
cirq.bloch_vector_from_state_vector(result.final_state, 2)

array([ 0.1048778 , -0.87166107,  0.4787564 ], dtype=float32)